# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [112]:
import warnings
warnings.filterwarnings('ignore')

In [113]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [114]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [115]:
data = {'Product': {0: 'Queen Size Sheet Set',
  1: 'Waterproof Phone Pouch',
  2: 'Luxury Air Mattress',
  3: 'Pillows Insert',
  4: 'Milk Frother Handheld\n',
  5: "L'Or Espresso Café\xa0\n",
  6: 'Hervidor de Agua Eléctrico'},
 'Review': {0: 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.',
  1: 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
  2: "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
  3: 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
  4: "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
  5: "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
  6: 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.'}}

In [116]:

import pandas as pd
df = pd.DataFrame(data)

In [117]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [118]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [119]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [120]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [121]:
chain = LLMChain(llm=llm, prompt=prompt)

In [122]:
product = "Queen Size Sheet Set"
chain.run(product)

'Queen Comfort Co.'

## SimpleSequentialChain

In [123]:
from langchain.chains import SimpleSequentialChain

In [124]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [125]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [126]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [127]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens
Royal Linens is a premium bedding and linen company that produces luxurious and comfortable products with attention to detail and quality.

> Finished chain.


'Royal Linens is a premium bedding and linen company that produces luxurious and comfortable products with attention to detail and quality.'

## SequentialChain

In [128]:
from langchain.chains import SequentialChain

In [129]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [130]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [131]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [132]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [133]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [134]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam doesn\'t hold, it\'s weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"',
 'summary': "The reviewer found the taste of the product to be mediocre and suspected that it may be an old batch or counterfeit as the foam didn't hold well.",
 'followup_message': "Réponse : Le critique a trouvé le goût du produit moyen et a suspecté qu'il pourrait s'agir d'un lot ancien ou contrefait car la mousse ne tenait pas bien."}

## RouterChain

In [135]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [136]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [137]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [138]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [139]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [140]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [141]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [142]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [143]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [144]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [145]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...


math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can answer this question easily. The answer to 2 + 2 is 4.'

In [146]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for the synthesis of proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next. Therefore, every cell in our body needs DNA to carry out its specific functions and to maintain the integrity of the organism as a whole.'

## Experiment on your own

In [147]:
# LLMChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)
prompt = ChatPromptTemplate.from_template(
    "What type of cuisine is {city} most famous for? Return a one word response."
)
chain = LLMChain(llm=llm, prompt=prompt)
city_1 = 'San Diego'
city_2 = 'New York'
city_3 = 'Jacksonville'
print(chain.run(city_1))
print(chain.run(city_2))
print(chain.run(city_3))

Mexican.
Bagels.
Seafood.


In [150]:
# SimpleSequentialChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)

first_prompt = ChatPromptTemplate.from_template(
    "What type of cuisine is {city} most famous for? Return a one word response."
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    "What are five popular dishes in {cusine} food?"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

third_prompt = ChatPromptTemplate.from_template(
    "Write a poem about the following dishes: {popular_dishes}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two, chain_three],
                                             verbose=True
                                            )

overall_simple_chain.run("San Diego")



> Entering new SimpleSequentialChain chain...
Mexican.
1. Tacos
2. Enchiladas
3. Quesadillas
4. Guacamole
5. Chiles Rellenos
Tacos, oh how I adore
The way you fill me up and more
Your crispy shells and seasoned beef
Are all I need to find relief

Enchiladas, wrapped with care
Tender meat and sauce so rare
Baked to perfection, cheese on top
You make my taste buds never stop

Quesadillas, ooey and gooey
Melting cheese and chicken truly
Folded in a tortilla nice and warm
Making my hunger forever calm

Guacamole, green and fresh
A perfect dip, I must confess
Ripe avocados and spicy kick
My heart you've always managed to pick

Chiles Rellenos, stuffed with flair
Peppers and cheese, a lovely pair
Crispy on the outside, soft within
You're a dish I can't help but grin

Oh Mexican cuisine, how you delight
My senses with every bite
Tacos, enchiladas, quesadillas galore
Guacamole and chiles rellenos, you are truly adored.

> Finished chain.


"Tacos, oh how I adore\nThe way you fill me up and more\nYour crispy shells and seasoned beef\nAre all I need to find relief\n\nEnchiladas, wrapped with care\nTender meat and sauce so rare\nBaked to perfection, cheese on top\nYou make my taste buds never stop\n\nQuesadillas, ooey and gooey\nMelting cheese and chicken truly\nFolded in a tortilla nice and warm\nMaking my hunger forever calm\n\nGuacamole, green and fresh\nA perfect dip, I must confess\nRipe avocados and spicy kick\nMy heart you've always managed to pick\n\nChiles Rellenos, stuffed with flair\nPeppers and cheese, a lovely pair\nCrispy on the outside, soft within\nYou're a dish I can't help but grin\n\nOh Mexican cuisine, how you delight\nMy senses with every bite\nTacos, enchiladas, quesadillas galore\nGuacamole and chiles rellenos, you are truly adored."

In [152]:
# SequentialChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)

first_prompt = ChatPromptTemplate.from_template(
    "Write a three sentence description of the following business:"
    "\n\n{business}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="business_description"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Define the primary age and gender demographic target audience for the following business:"
    "\n\n{business_description}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="demographics"
                    )

third_prompt = ChatPromptTemplate.from_template(
    "Create a three sentence description for a target audience of {business}, "
    "that is described by {business_description}, "
    "where the target audience's demographics are as follows: {demographics}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="target_audience"
                      )

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a facebook advertisement for {business}"
    "who is trying to reach the following target audience: {target_audience}"
    "that is of the following demographics {demographics}."
    "Make sure to use emojis in the post."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="facebook_post"
                     )

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["business"],
    output_variables=["business_description", "demographics", "target_audience", "facebook_post"],
    verbose=True
)


overall_chain("Nike")



> Entering new SequentialChain chain...

> Finished chain.


{'business': 'Nike',
 'business_description': "Nike is a multinational corporation that designs, develops, and sells athletic apparel, footwear, equipment, and accessories. It is one of the world's largest suppliers of athletic shoes and apparel and has partnerships with numerous professional athletes and sports teams. Nike has a strong brand identity and is known for its innovative designs, marketing campaigns, and commitment to sustainability.",
 'demographics': "The primary age and gender demographic target audience for Nike is typically between 18-40 years old and includes both male and female consumers who are interested in athletics, sports, fitness, and lifestyle products. Nike's products are also marketed towards athletes, sports enthusiasts, and active individuals who are inspired by the company's brand mission and values.",
 'target_audience': 'Nike caters to young and active adults who value physical activity, sports, and fashion. With a wide range of athletic apparel, footw

In [154]:
# RouterChain

fitness_trainer_template = """You are an experienced fitness trainer. \
You possess a deep understanding of human anatomy, physiology, and the principles of fitness and nutrition. \
You're skilled in creating personalized workout plans that are effective, safe, and tailored to individual needs and goals. \
Your expertise also includes motivation and lifestyle coaching to help individuals maintain a healthy, active lifestyle. \
You have the patience and communication skills to explain exercises clearly and to encourage progress, while always being mindful of safety and individual limitations.

Here is a question:
{input}"""

outdoor_enthusiast_template = """You are a passionate outdoor enthusiast. \
You have extensive knowledge of various outdoor activities, including hiking, camping, mountain biking, and kayaking. \
Your experiences have equipped you with a wealth of information about outdoor gear, survival skills, and environmental conservation. \
You understand the importance of respecting nature and leaving no trace behind. \
Your enthusiasm is infectious, and you're excellent at providing tips, tricks, and advice for others looking to explore and enjoy the great outdoors.

Here is a question:
{input}"""

travel_agent_template = """You are a seasoned travel agent. \
You possess a vast knowledge of global destinations, cultures, and travel logistics. \
Your expertise includes planning itineraries, understanding the best travel deals, and providing tailored recommendations for various types of travelers. \
You're skilled in making travel stress-free and enjoyable, taking into account clients' preferences, budget, and unique needs. \
Your passion for travel and commitment to customer satisfaction make you an invaluable guide in creating memorable travel experiences.

Here is a question:
{input}"""

machine_learning_expert_template = """You are a highly skilled machine learning expert. \
You have a deep understanding of algorithms, data structures, statistics, and programming. \
Your expertise encompasses various aspects of machine learning, including supervised and unsupervised learning, neural networks, and deep learning. \
You're adept at analyzing complex data sets, identifying patterns, and applying machine learning techniques to solve real-world problems. \
You can clearly explain complex concepts and are always up-to-date with the latest advancements in the field.

Here is a question:
{input}"""



prompt_infos = [
    {
        "name": "fitness trainer",
        "description": "Good for answering fitness and health questions",
        "prompt_template": fitness_trainer_template
    },
    {
        "name": "outdoor enthusiast",
        "description": "Good for answering questions about outdoor activities",
        "prompt_template": outdoor_enthusiast_template
    },
    {
        "name": "travel agent",
        "description": "Good for answering travel and vacation-related questions",
        "prompt_template": travel_agent_template
    },
    {
        "name": "machine learning expert",
        "description": "Good for answering questions about machine learning",
        "prompt_template": machine_learning_expert_template
    }
]


llm = ChatOpenAI(temperature=0, model=llm_model)


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )


print(chain.run("What are some good back exercises?"))
print(chain.run("Where is the longest hiking trail in the US?"))
print(chain.run("Which Hawaiian island is best for whale watching?"))
print(chain.run("Explain logisitic regression in one sentence."))
print(chain.run("What day of the year is Memorial Day?"))



> Entering new MultiPromptChain chain...
fitness trainer: {'input': 'What are some good back exercises?'}
> Finished chain.
There are many effective back exercises that can help strengthen and tone the muscles in your back. Some of the best exercises include:

1. Pull-ups or chin-ups
2. Lat pulldowns
3. Seated cable rows
4. Bent-over barbell rows
5. T-bar rows
6. Dumbbell rows
7. Back extensions
8. Deadlifts
9. Good mornings
10. Reverse flyes

It's important to choose exercises that target all areas of your back, including your upper, middle, and lower back muscles. Varying your exercises and incorporating different equipment can also help keep your workouts challenging and effective. As always, be sure to use proper form and start with lighter weights before progressing to heavier weights to avoid injury.


> Entering new MultiPromptChain chain...
outdoor enthusiast: {'input': 'Where can I find information about the longest hiking trail in the US?'}
> Finished chain.
The longest hik